# Mass Spectrometry Peak Annotation

In [1]:
import pandas as pd
import numpy as np

### Import Human Metabolomic DataBase Reference Dataset 

In [2]:
df = pd.read_csv('/QRISdata/Q1851/Andrew_C/Metabolomics/hmdb.csv')
df

/scratch/temp/6071631/ipykernel_428439/2823814329.py:1: DtypeWarning: Columns (6,8,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/QRISdata/Q1851/Andrew_C/Metabolomics/hmdb.csv')


,accession,monisotopic_molecular_weight,iupac_name,name,chemical_formula,InChIKey,cas_registry_number,smiles,drugbank,chebi_id,...,bigg,metlin_id,pdb_id,logpexp,kingdom,direct_parent,super_class,class,sub_class,molecular_framework
0,HMDB0000001,169.085127,b'(2S)-2-amino-3-(1-methyl-1H-imidazol-4-yl)pr...,b'1-Methylhistidine',C7H11N3O2,BRMWTNUJHUMWMS-LURJTMIESA-N,332-80-9,CN1C=NC(C[C@H](N)C(O)=O)=C1,DB04151,50599.0,...,NaN,3741.0,NaN,NaN,Organic compounds,Histidine and derivatives,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues",Aromatic heteromonocyclic compounds
1,HMDB0000002,74.084398,"b'propane-1,3-diamine'","b'1,3-Diaminopropane'",C3H10N2,XFNJVJPLKCPIBV-UHFFFAOYSA-N,109-76-2,NCCCN,NaN,15725.0,...,NaN,NaN,NaN,-1.43,Organic compounds,Monoalkylamines,Organic nitrogen compounds,Organonitrogen compounds,Amines,Aliphatic acyclic compounds
2,HMDB0000005,102.031694,b'2-oxobutanoic acid',b'2-Ketobutyric acid',C4H6O3,TYEYBOSBBBHJIV-UHFFFAOYSA-N,600-18-0,CCC(=O)C(O)=O,DB04553,30831.0,...,33889.0,NaN,NaN,NaN,Organic compounds,Short-chain keto acids and derivatives,Organic acids and derivatives,Keto acids and derivatives,Short-chain keto acids and derivatives,Aliphatic acyclic compounds
3,HMDB0000008,104.047344,b'(2S)-2-hydroxybutanoic acid',b'2-Hydroxybutyric acid',C4H8O3,AFENDNXGAFYKQO-VKHMYHEASA-N,3347-90-8,CC[C@H](O)C(O)=O,NaN,50613.0,...,NaN,NaN,NaN,NaN,Organic compounds,Alpha hydroxy acids and derivatives,Organic acids and derivatives,Hydroxy acids and derivatives,Alpha hydroxy acids and derivatives,Aliphatic acyclic compounds
4,HMDB0000010,300.172545,"b'(1S,10R,11S,15S)-5-hydroxy-4-methoxy-15-meth...",b'2-Methoxyestrone',C19H24O3,WHEUWNKSCXYKBU-QPWUGHHJSA-N,362-08-3,[H][C@@]12CCC(=O)[C@@]1(C)CC[C@]1([H])C3=C(CC[...,NaN,1189.0,...,NaN,2578.0,NaN,NaN,Organic compounds,Estrogens and derivatives,Lipids and lipid-like molecules,Steroids and steroid derivatives,Estrane steroids,Aromatic homopolycyclic compounds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217915,HMDB0304947,378.277010,"b'3-{5,16-dihydroxy-2,15-dimethyltetracyclo[8....",b'Nordeoxycholic acid',C23H38O4,PLRQOCVIINWCFA-UHFFFAOYSA-N,NaN,CC(CC(O)=O)C1CCC2C3CCC4CC(O)CCC4(C)C3CC(O)C12C,NaN,NaN,...,NaN,NaN,NaN,NaN,Organic compounds,"Dihydroxy bile acids, alcohols and derivatives",Lipids and lipid-like molecules,Steroids and steroid derivatives,"Bile acids, alcohols and derivatives",Aliphatic homopolycyclic compounds
217916,HMDB0304950,374.282095,"b'4-{2,15-dimethyl-5-oxotetracyclo[8.7.0.0^{2,...",b'3-Oxo-5beta-cholanoic acid',C24H38O3,KIQFUORWRVZTHT-UHFFFAOYSA-N,NaN,CC(CCC(O)=O)C1CCC2C3CCC4CC(=O)CCC4(C)C3CCC12C,NaN,NaN,...,NaN,NaN,NaN,NaN,Organic compounds,"Bile acids, alcohols and derivatives",Lipids and lipid-like molecules,Steroids and steroid derivatives,"Bile acids, alcohols and derivatives",Aliphatic homopolycyclic compounds
217917,HMDB0304951,302.245710,"b'2,3-dihydroxypropyl tetradecanoate'",b'Glycerol 1-myristate',C17H34O4,DCBSHORRWZKAKO-UHFFFAOYSA-N,NaN,CCCCCCCCCCCCCC(=O)OCC(O)CO,NaN,75562.0,...,NaN,NaN,NaN,NaN,Organic compounds,1-monoacylglycerols,Lipids and lipid-like molecules,Glycerolipids,Monoradylglycerols,Aliphatic acyclic compounds
217918,HMDB0304953,173.998680,b'2-hydroxybenzene-1-sulfonic acid',b'O-Phenolsulfonic acid',C6H6O4S,IULJSGIJJZZUMF-UHFFFAOYSA-N,NaN,OC1=CC=CC=C1S(O)(=O)=O,NaN,71049.0,...,NaN,NaN,NaN,NaN,Organic compounds,Benzenesulfonic acids and derivatives,Benzenoids,Benzene and substituted derivatives,Benzenesulfonic acids and derivatives,Aromatic homomonocyclic compounds


### Import Significant Peaks

In [9]:
all_mz = pd.read_csv('/QRISdata/Q1851/Andrew_C/Metabolomics/DEGs.csv')
all_mz = all_mz.drop(["Unnamed: 0"], axis=1)

#### Extract m/z values from gene string
all_mz["mz"] = all_mz['gene'].str.replace('mz-', '').astype(float)
all_mz

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene,mz
0,0.000000e+00,43.934340,0.954,0.938,0.000000e+00,Control,mz-422.928788107395,422.928788
1,0.000000e+00,38.476651,0.968,0.961,0.000000e+00,Control,mz-982.896490742609,982.896491
2,0.000000e+00,33.265975,0.992,0.978,0.000000e+00,Control,mz-598.935441543004,598.935442
3,0.000000e+00,28.795980,0.994,0.983,0.000000e+00,Control,mz-790.917619651681,790.917620
4,7.980825e-308,28.875012,0.993,0.992,5.073330e-303,Control,mz-614.908708454546,614.908708
...,...,...,...,...,...,...,...,...
1959,7.021438e-03,2.199718,0.282,0.269,1.000000e+00,Treated,mz-875.560116983688,875.560117
1960,7.915133e-03,3.772457,0.276,0.259,1.000000e+00,Treated,mz-853.489991081482,853.489991
1961,8.066559e-03,0.381883,0.422,0.387,1.000000e+00,Treated,mz-835.594807240167,835.594807
1962,8.131028e-03,10.825634,0.706,0.688,1.000000e+00,Treated,mz-800.433787633646,800.433788


### Run Peak Annotation Label Transfer

In [10]:
### Below Script find the closest peak in the annotation set to each of the peaks in the query dataset
closest_annotations = {}
df_weights = df["monisotopic_molecular_weight"].values  # Convert the target column to a NumPy array

for row in all_mz.index:
    if row % 100 == 0:
        print(f"Calculating value for index {row}...")
    mz = all_mz["mz"][row]
    differences = np.abs(df_weights - mz)  # Calculate absolute differences in one go
    min_value = min(differences)  # Find the index of the minimum difference
    min_idx = list(differences).index(min_value)
    closest_annotations[mz] = (min_idx, min_value, list(df["name"].values)[min_idx])

Calculating value for index 0...
Calculating value for index 100...
Calculating value for index 200...
Calculating value for index 300...
Calculating value for index 400...
Calculating value for index 500...
Calculating value for index 600...
Calculating value for index 700...
Calculating value for index 800...
Calculating value for index 900...
Calculating value for index 1000...
Calculating value for index 1100...
Calculating value for index 1200...
Calculating value for index 1300...
Calculating value for index 1400...
Calculating value for index 1500...
Calculating value for index 1600...
Calculating value for index 1700...
Calculating value for index 1800...
Calculating value for index 1900...


In [11]:
### Manipulate Dataframe to merge with original
annotations = pd.DataFrame.from_dict(closest_annotations, orient='index', columns=['Reference_Dataset_Index', 'm/z_Difference', 'Metabolite'])
annotated_dataset = all_mz

annotations["mz"] = annotations.index
annotations.index
annotations.reset_index(drop=True, inplace=True)

In [12]:
## Update Query Dataset
annotated_dataset["Metabolite"] = annotations["Metabolite"]
annotated_dataset["m/z_Difference"] = annotations["m/z_Difference"]
annotated_dataset["Reference_Dataset_Index"] = annotations["Reference_Dataset_Index"]

### Add Threshold for Correct Annotations
annotated_dataset['Accurate_Annotation_(<=0.005)'] = annotated_dataset['m/z_Difference'] <= 0.005

In [13]:
annotated_dataset

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene,mz,Metabolite,m/z_Difference,Reference_Dataset_Index,Accurate_Annotation_(<=0.005)
0,0.000000e+00,43.934340,0.954,0.938,0.000000e+00,Control,mz-422.928788107395,422.928788,b'Gemcitabine diphosphate',0.075635,40632,False
1,0.000000e+00,38.476651,0.968,0.961,0.000000e+00,Control,mz-982.896490742609,982.896491,"b'TG(15:0/24:1(15Z)/22:2(13Z,16Z))'",0.003649,23815,True
2,0.000000e+00,33.265975,0.992,0.978,0.000000e+00,Control,mz-598.935441543004,598.935442,b'Adtepp',0.220892,163000,False
3,0.000000e+00,28.795980,0.994,0.983,0.000000e+00,Control,mz-790.917619651681,790.917620,"b""Isophthalamide, N,N'-bis(2,3-dihydroxypropyl...",0.047870,164258,False
4,7.980825e-308,28.875012,0.993,0.992,5.073330e-303,Control,mz-614.908708454546,614.908708,b'Idasanutlin',0.241610,167552,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1959,7.021438e-03,2.199718,0.282,0.269,1.000000e+00,Treated,mz-875.560116983688,875.560117,"b'PE(22:2(13Z,16Z)/22:6(5Z,8E,10Z,13Z,15E,19Z)...",0.007518,196975,False
1960,7.915133e-03,3.772457,0.276,0.259,1.000000e+00,Treated,mz-853.489991081482,853.489991,"b'PS(20:5(5Z,8Z,11Z,14Z,17Z)/22:6(4Z,7Z,10Z,13...",0.000607,91954,True
1961,8.066559e-03,0.381883,0.422,0.387,1.000000e+00,Treated,mz-835.594807240167,835.594807,b'DG(22:0/LTE4/0:0)',0.004783,209699,True
1962,8.131028e-03,10.825634,0.706,0.688,1.000000e+00,Treated,mz-800.433787633646,800.433788,"b'PGP(a-13:0/18:1(12Z)-O(9S,10R))'",0.009707,187307,False


In [14]:
### Save Dataset
annotated_dataset.to_csv("/QRISdata/Q1851/Andrew_C/Metabolomics/annotated_DEPs.csv")


In [16]:
annotated_dataset[annotated_dataset["cluster"] == "Treated"][1:10]

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene,mz,Metabolite,m/z_Difference,Reference_Dataset_Index,Accurate_Annotation_(<=0.005)
1518,0.0,53.426283,0.907,0.884,0.0,Treated,mz-770.503205666763,770.503206,"b'PG(16:0/20:4(5Z,8Z,11Z,14Z))'",0.006579,6542,False
1519,0.0,33.772458,0.989,0.987,0.0,Treated,mz-768.494814390676,768.494814,"b'PG(20:4(5Z,8Z,11Z,14Z)/16:1(9Z))'",0.000679,95858,True
1520,0.0,19.697506,0.387,0.004,0.0,Treated,mz-450.259379341302,450.259379,b'DG(2:0/PGJ2/0:0)',0.002374,209903,True
1521,0.0,16.582380,0.268,0.004,0.0,Treated,mz-450.263881957609,450.263882,b'DG(2:0/PGJ2/0:0)',0.002128,209903,True
1522,0.0,16.510586,0.997,0.996,0.0,Treated,mz-771.513226285263,771.513226,"b'PS(15:0/20:3(5Z,8Z,11Z))'",0.008192,91579,False
1523,0.0,14.956284,0.302,0.002,0.0,Treated,mz-448.246729024761,448.246729,b'3-Hydroxy-3-methylglutarylcarnitine',0.004648,161058,True
1524,0.0,14.944664,0.306,0.004,0.0,Treated,mz-448.242246579883,448.242247,b'3-Hydroxy-3-methylglutarylcarnitine',0.000166,161058,True
1525,0.0,12.954875,0.910,0.802,0.0,Treated,mz-766.47632850122,766.476329,"b'PG(18:3(6Z,9Z,12Z)/18:3(6Z,9Z,12Z))'",0.002157,6628,True
1526,0.0,8.557970,0.992,0.987,0.0,Treated,mz-732.551624288964,732.551624,"b'DG(21:0/20:5(7Z,9Z,11E,13E,17Z)-3OH(5,6,15)/...",0.002395,209563,True
